In [98]:
import os
import sys
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set up the directories
code_dir = os.getcwd()
project_dir = os.path.dirname(code_dir)
data_dir = os.path.join(project_dir, 'data')

# Read data
nutritional_requirements_df = pd.read_excel(os.path.join(data_dir, 'female_nutritional_requirements.xlsx'), index_col=0, header=0)
food_items_cost_co2_intake_df = pd.read_excel(os.path.join(data_dir, 'food_items_cost_co2_intake.xlsx'), index_col=2, header=0)
food_items_nutritional_composition_df = pd.read_excel(os.path.join(data_dir, 'food_items_nutritional_composition.xlsx'), index_col=2, header=0)
food_groups_mean_intake_df = pd.read_excel(os.path.join(data_dir, 'food_groups_mean_intake.xlsx'), index_col=1, header=0)

# Data for the model
nutrients = list(nutritional_requirements_df.index)
nutrients.remove('Pantothenic acid (mg)')

food_items = list(food_items_cost_co2_intake_df.index)
food_groups = list(food_items_cost_co2_intake_df['Food group'].unique())

# Nutritional requirements dictionary (min and max intake)
nutritional_requirements = {nutrient : {} for nutrient in nutrients}
for nutrient in nutrients:
    rni_value = nutritional_requirements_df.loc[nutrient, 'RNI']
    nutritional_requirements[nutrient]['min intake'] = np.float64(0) if pd.isna(rni_value) else rni_value
    ul_value = nutritional_requirements_df.loc[nutrient, 'UL']
    nutritional_requirements[nutrient]['max intake'] = np.inf if pd.isna(ul_value) else ul_value

# Food items data dictionary (cost, co2, min and max intake)
food_items_data = {food_item : {} for food_item in food_items}
for food_item in food_items:
    food_item_dict = {}
    food_item_dict['cost'] = food_items_cost_co2_intake_df.loc[food_item, 'Cost (KHR/g)']
    food_item_dict['co2'] = food_items_cost_co2_intake_df.loc[food_item, 'CO2e (Kg/1000g)']
    food_item_dict['min intake'] = food_items_cost_co2_intake_df.loc[food_item, 'Min Intake (g)']
    food_item_dict['max intake'] = food_items_cost_co2_intake_df.loc[food_item, 'Max Intake (g)']
    food_items_data[food_item] = food_item_dict

# Food items nutrients dictionary (nutritional composition)
food_items_nutrients = {food_item : {} for food_item in food_items}
for food_item in food_items:
    food_item_dict = {}
    for nutrient in nutrients:
        food_item_dict[nutrient] = food_items_nutritional_composition_df.loc[food_item, nutrient]
    food_items_nutrients[food_item] = food_item_dict

# Food groups dictionary (mean intake and food items list)
food_groups_data = {food_group : {} for food_group in food_groups}
for food_group in food_groups:
    food_group_dict = {}
    food_group_dict['mean intake'] = food_groups_mean_intake_df.loc[food_group, 'Mean Intake (g)']
    food_group_dict['food items'] = list(food_items_cost_co2_intake_df[food_items_cost_co2_intake_df['Food group'] == food_group].index)
    food_groups_data[food_group] = food_group_dict

In [67]:
# Create the model
model = pulp.LpProblem('FoodBasketThreeObjectivesLP', pulp.LpMinimize)

# Decision variables
food_items_vars = pulp.LpVariable.dicts('FoodItem', food_items, lowBound=0, cat='Continuous')

# Objectives
f1 = pulp.LpVariable('Cost', lowBound=0, cat='Continuous') # Minimize cost
f2 = pulp.LpVariable('Deviation from current consumption', lowBound=0, cat='Continuous') # Maximize energy intake
f3 = pulp.LpVariable('CO2e', lowBound=0, cat='Continuous') # Minimize CO2 emissions


# List of variables AND objectives
variables = [food_items_vars[food_item] for food_item in food_items]
objectives = [f1, f2, f3]


In [70]:
food_items_cost_co2_intake_df


,food_id,group_id,Food group,Cost (KHR/g),CO2e (Kg/1000g),Min Intake (g),Max Intake (g)
Food item,,,,,,,
"Noodles, egg, dry, enriched",1,1,Grains & cereals,8.72,0.41,0,394.40
"Noodles, rice, cooked",2,1,Grains & cereals,2.44,1.37,0,1402.31
"Rice, white, polished, raw",3,1,Grains & cereals,2.12,1.32,0,429.04
"Cassava, cooked",4,2,Roots,3.51,0.35,0,889.31
"Cassava, white",5,2,Roots,2.12,0.35,0,889.31
...,...,...,...,...,...,...,...
"Butter, CotD",83,10,Oils & fats,24.54,11.31,0,98.45
"Oil, vegetable",84,10,Oils & fats,8.63,0.16,0,81.09
"Sugar, palm",85,11,Sugar,4.62,1.58,0,31.57
